In [1]:
import os
import subprocess
import uproot
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm

# Extraction and Reduction

In [2]:
def is_file_corrupt(file_path):
    try:
        with uproot.open(file_path) as file:
            file.keys()
        return False
    except:
        return True

In [3]:
def process_file(args):
    filename, input_folder, output_folder, decay_tree_line, branches_to_keep = args
    input_file_path = os.path.join(input_folder, filename)
    output_file_path = os.path.join(output_folder, filename.replace('.root', '_reduced.root'))

    if os.path.exists(output_file_path):
        return f"Skipped existing file: {output_file_path}"
    if is_file_corrupt(input_file_path):
        return f"File {filename} seems to be corrupt. Skipping."

    try:
        with uproot.open(input_file_path) as file:
            if decay_tree_line not in file:
                return f"Tree {decay_tree_line} not found in {filename}. Skipping."

            tree = file[decay_tree_line]
            data = tree.arrays(expressions=branches_to_keep, library="np")
            
            # Number of events before pre-selection
            num_events_before = len(data['Bu_MM'])
            
            # Calculations for pre-selection
            Delta_Z = data['L0_ENDVERTEX_Z'] - data['Bu_ENDVERTEX_Z']
            Delta_X = data['L0_ENDVERTEX_X'] - data['Bu_ENDVERTEX_X']
            Delta_Y = data['L0_ENDVERTEX_Y'] - data['Bu_ENDVERTEX_Y']

            Delta_X_ERR = np.sqrt(np.square(data['Bu_ENDVERTEX_XERR']) + np.square(data['L0_ENDVERTEX_XERR']))
            Delta_Y_ERR = np.sqrt(np.square(data['Bu_ENDVERTEX_YERR']) + np.square(data['L0_ENDVERTEX_YERR']))
            Delta_Z_ERR = np.sqrt(np.square(data['Bu_ENDVERTEX_ZERR']) + np.square(data['L0_ENDVERTEX_ZERR']))

            delta_x = np.divide(Delta_X, Delta_X_ERR, out=np.zeros_like(Delta_X), where=Delta_X_ERR!=0)
            delta_y = np.divide(Delta_Y, Delta_Y_ERR, out=np.zeros_like(Delta_Y), where=Delta_Y_ERR!=0)
            delta_z = np.divide(Delta_Z, Delta_Z_ERR, out=np.zeros_like(Delta_Z), where=Delta_Z_ERR!=0)

            L0_FD_CHISQ = np.square(delta_x) + np.square(delta_y) + np.square(delta_z)

            prodProbKK = np.multiply(data['h1_ProbNNk'], data['h2_ProbNNk'])

            # Apply pre-selection
            pre_select = (data['Bu_FDCHI2_OWNPV'] > 175) & (Delta_Z > 2.5) & \
                         (data['Lp_ProbNNp'] > 0.05) & (data['p_ProbNNp'] > 0.05) & \
                         (prodProbKK > 0.10) & (data['Bu_DTF_chi2'] < 30) & (data['Bu_PT'] > 3000)
            data = {branch: values[pre_select] for branch, values in data.items()}

            # Number of events after pre-selection
            num_events_after = len(data['Bu_MM'])
            
            # Percentage of events remaining
            percent_remaining = (num_events_after / num_events_before) * 100 if num_events_before > 0 else 0

            with uproot.recreate(output_file_path) as new_file:
                new_file[decay_tree_line] = data

            return f"Processed {filename}: {num_events_before} events before, {num_events_after} after ({percent_remaining:.2f}% remaining)"
    except Exception as e:
        return f"Error processing {filename}: {e}"


In [4]:
def main(year, magnet, decay_tree, ll_or_dd, num_files=None):
    
    # Extract the base name from the decay tree (e.g., "B2L0barPKpKm" from "B2L0barPKpKm_LL")
    decay_tree_base = decay_tree.split('_')[0]

    # Check for consistency between decay_tree and ll_or_dd arguments
    if ('_LL/' in decay_tree and ll_or_dd != 'LL') or ('_DD/' in decay_tree and ll_or_dd != 'DD'):
        raise ValueError("Inconsistency in decay_tree and ll_or_dd. Both should be LL or DD.")
        
    input_folder = f"/eos/lhcb/wg/BnoC/Bu2LambdaPPP/RD/restripped.data/reduced/{year}{magnet}/"
    output_folder = f"/eos/lhcb/user/m/melashri/data/red_RD/{decay_tree_base}/{year}/{magnet}/{ll_or_dd}"

    # Print the output folder location
    print(f"Output files will be saved in: {output_folder}")
    
    # Check if the output folder exists, otherwise create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # The list of branches to keep
    branches_to_keep = ['Bu_MM', 'Bu_MMERR', 'Bu_ID', 'Bu_P', 'Bu_PT', 'Bu_PE', 'Bu_PX', 'Bu_PY', 'Bu_PZ', 
                    'h1_P', 'h1_PT','h1_PE', 'h1_PX', 'h1_PY', 'h1_PZ', 
                    'h1_ID', 'h1_TRACK_Type', 'h1_IPCHI2_OWNPV', 'h1_PIDK', 'h1_PIDp',
                    'h2_P', 'h2_PT', 'h2_PE', 'h2_PX','h2_PY', 'h2_PZ', 
                    'h2_ID', 'h2_TRACK_Type', 'h2_IPCHI2_OWNPV', 'h2_PIDK', 'h2_PIDp',
                    'p_P', 'p_PT', 'p_PE', 'p_PX', 'p_PY', 'p_PZ', 
                    'p_ID', 'p_TRACK_Type', 'p_IPCHI2_OWNPV', 'p_PIDK', 'p_PIDp',
                    'Lp_P', 'Lp_PT', 'Lp_PE', 'Lp_PX', 'Lp_PY', 'Lp_PZ', 'Lp_ID', 'Lp_TRACK_Type', 'Lp_PIDK', 'Lp_PIDp', 
                    'L0_P',
                    'L0_PT', 'L0_PE', 'L0_PX', 'L0_PY', 'L0_PZ', 'L0_ID', 'Lpi_P', 'Lpi_PT', 'Lpi_PE', 'Lpi_PX', 'Lpi_PY', 
                    'Lpi_PZ', 'Lpi_ID', 'Lpi_TRACK_Type', 'p_ProbNNp', 'Lpi_ProbNNpi', 'Lp_ProbNNp', 'h1_ProbNNk', 'h2_ProbNNk',
                    'Bu_FDCHI2_OWNPV', 'Bu_IPCHI2_OWNPV', 'Bu_ENDVERTEX_X', 'Bu_ENDVERTEX_Y', 'Bu_ENDVERTEX_Z', 'Bu_ENDVERTEX_XERR', 
                    'Bu_ENDVERTEX_YERR', 'Bu_ENDVERTEX_ZERR', 'Bu_DTF_decayLength', 'Bu_DTF_decayLengthErr', 'Bu_DTF_ctau', 
                    'Bu_DTF_ctauErr', 'Bu_DTF_status', 'Bu_DTFL0_M', 'Bu_DTFL0_MERR', 'Bu_DTFL0_ctau', 'Bu_DTFL0_ctauErr', 'Bu_DTF_nPV',
                    'Bu_DTF_chi2', 'Bu_DTF_nDOF', 'L0_ENDVERTEX_X', 'L0_ENDVERTEX_Y', 'L0_ENDVERTEX_Z', 'L0_ENDVERTEX_XERR', 
                    'L0_ENDVERTEX_YERR', 'L0_ENDVERTEX_ZERR', 'L0_MM', 'L0_OWNPV_Z', 'L0_OWNPV_ZERR', 'L0_FD_OWNPV', 'L0_FDCHI2_OWNPV',
                    'Bu_L0Global_TIS', 'Bu_L0HadronDecision_TOS', 'Bu_Hlt1Global_TIS', 'Bu_Hlt1TrackMVADecision_TOS',
                    'Bu_Hlt1TwoTrackMVADecision_TOS', 'Bu_Hlt2Topo2BodyDecision_TOS', 'Bu_Hlt2Topo3BodyDecision_TOS', 'Bu_Hlt2Topo4BodyDecision_TOS',
                    'Bu_Hlt2Topo2BodyBBDTDecision_TOS', 'Bu_Hlt2Topo3BodyBBDTDecision_TOS', 'Bu_Hlt2Topo4BodyBBDTDecision_TOS',
                    'h1_PIDK', 'h1_PIDp', 'h2_PIDK', 'h2_PIDp', 'p_PIDK', 'p_PIDp',
                    'Bu_DTFL0_chi2', 'Bu_DTFL0_nDOF', 'Bu_DTFL0_status',
                    'nCandidate','totCandidates', 'EventInSequence', 'runNumber', 'eventNumber','GpsTime',
                    'Polarity', 
                    'Bu_DTFL0_Lambda0_M', 'Bu_DTFL0_Lambda0_ctau', 'Bu_DTFL0_Lambda0_ctauErr',
                    'Bu_DTFL0_Lambda0_decayLength', 'Bu_DTFL0_Lambda0_decayLengthErr',
                    'Lp_ProbNNghost', 'Lpi_ProbNNghost', 'p_ProbNNghost', 'h1_ProbNNghost', 'h2_ProbNNghost',
                       ]
    # Go through all files the ends with .root 
    root_files = [f for f in os.listdir(input_folder) if f.endswith('.root')]
    
    # Go through only the first couple files that passed by num_files
    if num_files is not None:
        root_files = root_files[:num_files]
    
    # Arguments for the main function
    args = [(filename, input_folder, output_folder, decay_tree, branches_to_keep) for filename in root_files]

    # Use multiprocessing with 4 cores (max on SWAN)
    with Pool(4) as p:
        results = list(tqdm(p.imap(process_file, args), total=len(root_files), desc="Processing files"))

        for result in results:
            print(result)



## Run the Preprocessing

In [18]:
if __name__ == "__main__":
    year = "2017"
    magnet = "MagDown"
    decay_tree = "B2L0barPKpKm_LL/DecayTree"
    ll_or_dd = "LL"
    num_files = 16  # Process only the first 16 files for testing
    #main(year, magnet, decay_tree, ll_or_dd, num_files)
    main(year, magnet, decay_tree, ll_or_dd) # process all files


Output files will be saved in: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL


Processing files: 100%|██████████| 1434/1434 [11:42<00:00,  2.04it/s]

Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_0_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_1_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_10_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_100_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_1000_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_1001_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_1002_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDown/LL/datadata_2017MagDown_1003_reduced.root
Skipped existing file: /eos/lhcb/user/m/melashri/data/reduced/2017/MagDow

## Run over possible pre-defined configration

### B2L0barPKpKm

In [27]:
# Possible configurations
decays = ['B2L0barPKpKp']
decay_tree_lines = [f'{name}_{ending}' for name in decays for ending in ['LL', 'DD']]
years = ['2016', '2017']
magnet_polarizations = ['MagDown', 'MagUp']

print(f"The decay trees being processed are the following",decay_tree_lines)


# Set to a specific number for testing, or None for processing all files
num_files_for_testing = None  # or set to 16 for testing

# Iterate through all configurations and call main
for year in years:
    for magnet in magnet_polarizations:
        for decay_tree_line in decay_tree_lines:
            ll_or_dd = 'LL' if '_LL' in decay_tree_line else 'DD'
            decay_tree = f"{decay_tree_line}/DecayTree"

            # Call the main function with the current configuration
            # If num_files_for_testing is None, all files will be processed
            print(f"Processing: Year: {year}, Magnet: {magnet}, Decay Tree: {decay_tree}, Limit: {'All files' if num_files_for_testing is None else f'{num_files_for_testing} files'}")
            main(year, magnet, decay_tree, ll_or_dd, num_files=num_files_for_testing)

Processing: Year: 2016, Magnet: MagDown, Decay Tree: B2L0barPKpKm_LL/DecayTree, Limit: All files
Output files will be saved in: /eos/lhcb/user/m/melashri/data/red_RD/B2L0barPKpKm/2016/MagDown/LL


Processing files: 100%|██████████| 1625/1625 [12:04<00:00,  2.24it/s]


Processed datadata_2016MagDown_0.root: 342 events before, 211 after (61.70% remaining)
Processed datadata_2016MagDown_1.root: 436 events before, 231 after (52.98% remaining)
Processed datadata_2016MagDown_10.root: 413 events before, 254 after (61.50% remaining)
Processed datadata_2016MagDown_100.root: 432 events before, 256 after (59.26% remaining)
Processed datadata_2016MagDown_1000.root: 289 events before, 160 after (55.36% remaining)
Processed datadata_2016MagDown_1001.root: 376 events before, 219 after (58.24% remaining)
Processed datadata_2016MagDown_1002.root: 395 events before, 222 after (56.20% remaining)
Processed datadata_2016MagDown_1003.root: 349 events before, 212 after (60.74% remaining)
Processed datadata_2016MagDown_1004.root: 213 events before, 137 after (64.32% remaining)
Processed datadata_2016MagDown_1005.root: 325 events before, 178 after (54.77% remaining)
Processed datadata_2016MagDown_1006.root: 319 events before, 197 after (61.76% remaining)
Processed datadata_

Processing files: 100%|██████████| 1625/1625 [11:51<00:00,  2.28it/s]


Processed datadata_2016MagDown_0.root: 251 events before, 165 after (65.74% remaining)
Processed datadata_2016MagDown_1.root: 294 events before, 197 after (67.01% remaining)
Processed datadata_2016MagDown_10.root: 260 events before, 175 after (67.31% remaining)
Processed datadata_2016MagDown_100.root: 344 events before, 245 after (71.22% remaining)
Processed datadata_2016MagDown_1000.root: 207 events before, 148 after (71.50% remaining)
Processed datadata_2016MagDown_1001.root: 228 events before, 171 after (75.00% remaining)
Processed datadata_2016MagDown_1002.root: 257 events before, 173 after (67.32% remaining)
Processed datadata_2016MagDown_1003.root: 246 events before, 169 after (68.70% remaining)
Processed datadata_2016MagDown_1004.root: 134 events before, 103 after (76.87% remaining)
Processed datadata_2016MagDown_1005.root: 208 events before, 138 after (66.35% remaining)
Processed datadata_2016MagDown_1006.root: 183 events before, 123 after (67.21% remaining)
Processed datadata_

...

Processed datadata_2017MagDown_0.root: 229 events before, 173 after (75.55% remaining)
Processed datadata_2017MagDown_1.root: 306 events before, 220 after (71.90% remaining)
Processed datadata_2017MagDown_10.root: 290 events before, 210 after (72.41% remaining)
Processed datadata_2017MagDown_100.root: 271 events before, 194 after (71.59% remaining)
Processed datadata_2017MagDown_1000.root: 301 events before, 210 after (69.77% remaining)
Processed datadata_2017MagDown_1001.root: 289 events before, 215 after (74.39% remaining)
Processed datadata_2017MagDown_1002.root: 290 events before, 219 after (75.52% remaining)
Processed datadata_2017MagDown_1003.root: 333 events before, 238 after (71.47% remaining)
Processed datadata_2017MagDown_1004.root: 242 events before, 173 after (71.49% remaining)
Processed datadata_2017MagDown_1005.root: 314 events before, 229 after (72.93% remaining)
Processed datadata_2017MagDown_1006.root: 278 events before, 188 after (67.63% remaining)
Processed datadata_

Processing files: 100%|██████████| 1364/1364 [10:03<00:00,  2.26it/s]


Processed datadata_2017MagUp_0.root: 250 events before, 141 after (56.40% remaining)
Processed datadata_2017MagUp_1.root: 341 events before, 207 after (60.70% remaining)
Processed datadata_2017MagUp_10.root: 273 events before, 183 after (67.03% remaining)
Processed datadata_2017MagUp_100.root: 322 events before, 187 after (58.07% remaining)
Processed datadata_2017MagUp_1000.root: 326 events before, 207 after (63.50% remaining)
Processed datadata_2017MagUp_1001.root: 320 events before, 219 after (68.44% remaining)
Processed datadata_2017MagUp_1002.root: 361 events before, 212 after (58.73% remaining)
Processed datadata_2017MagUp_1003.root: 212 events before, 117 after (55.19% remaining)
Processed datadata_2017MagUp_1004.root: 424 events before, 251 after (59.20% remaining)
Processed datadata_2017MagUp_1005.root: 324 events before, 198 after (61.11% remaining)
Processed datadata_2017MagUp_1006.root: 335 events before, 194 after (57.91% remaining)
Processed datadata_2017MagUp_1007.root: 2

Processing files: 100%|██████████| 1364/1364 [10:00<00:00,  2.27it/s]

Processed datadata_2017MagUp_0.root: 161 events before, 103 after (63.98% remaining)
Processed datadata_2017MagUp_1.root: 301 events before, 216 after (71.76% remaining)
Processed datadata_2017MagUp_10.root: 239 events before, 147 after (61.51% remaining)
Processed datadata_2017MagUp_100.root: 256 events before, 174 after (67.97% remaining)
Processed datadata_2017MagUp_1000.root: 256 events before, 179 after (69.92% remaining)
Processed datadata_2017MagUp_1001.root: 223 events before, 152 after (68.16% remaining)
Processed datadata_2017MagUp_1002.root: 324 events before, 220 after (67.90% remaining)
Processed datadata_2017MagUp_1003.root: 178 events before, 120 after (67.42% remaining)
Processed datadata_2017MagUp_1004.root: 315 events before, 227 after (72.06% remaining)
Processed datadata_2017MagUp_1005.root: 260 events before, 169 after (65.00% remaining)
Processed datadata_2017MagUp_1006.root: 282 events before, 197 after (69.86% remaining)
Processed datadata_2017MagUp_1007.root: 2

### B2L0PbarKpKp

In [7]:
# Possible configurations
decays = ['B2L0PbarKpKp']
decay_tree_lines = [f'{name}_{ending}' for name in decays for ending in ['LL', 'DD']]
years = ['2016', '2017']
magnet_polarizations = ['MagDown', 'MagUp']

print(f"The decay trees being processed are the following",decay_tree_lines)

# Set to a specific number for testing, or None for processing all files
num_files_for_testing = None  # or set to 16 for testing

# Iterate through all configurations and call main
for year in years:
    for magnet in magnet_polarizations:
        for decay_tree_line in decay_tree_lines:
            ll_or_dd = 'LL' if '_LL' in decay_tree_line else 'DD'
            decay_tree = f"{decay_tree_line}/DecayTree"

            # Call the main function with the current configuration
            # If num_files_for_testing is None, all files will be processed
            print(f"Processing: Year: {year}, Magnet: {magnet}, Decay Tree: {decay_tree}, Limit: {'All files' if num_files_for_testing is None else f'{num_files_for_testing} files'}")
            main(year, magnet, decay_tree, ll_or_dd, num_files=num_files_for_testing)

The decay trees being processed are the following ['B2L0PbarKpKp_LL', 'B2L0PbarKpKp_DD']
Processing: Year: 2016, Magnet: MagDown, Decay Tree: B2L0PbarKpKp_LL/DecayTree, Limit: All files
Output files will be saved in: /eos/lhcb/user/m/melashri/data/red_RD/B2L0PbarKpKp/2016/MagDown/LL


Processing files: 100%|██████████| 1625/1625 [12:17<00:00,  2.20it/s]


Processed datadata_2016MagDown_0.root: 152 events before, 95 after (62.50% remaining)
Processed datadata_2016MagDown_1.root: 200 events before, 102 after (51.00% remaining)
Processed datadata_2016MagDown_10.root: 155 events before, 76 after (49.03% remaining)
Processed datadata_2016MagDown_100.root: 150 events before, 89 after (59.33% remaining)
Processed datadata_2016MagDown_1000.root: 130 events before, 82 after (63.08% remaining)
Processed datadata_2016MagDown_1001.root: 174 events before, 114 after (65.52% remaining)
Processed datadata_2016MagDown_1002.root: 213 events before, 122 after (57.28% remaining)
Processed datadata_2016MagDown_1003.root: 139 events before, 82 after (58.99% remaining)
Processed datadata_2016MagDown_1004.root: 92 events before, 55 after (59.78% remaining)
Processed datadata_2016MagDown_1005.root: 163 events before, 102 after (62.58% remaining)
Processed datadata_2016MagDown_1006.root: 141 events before, 81 after (57.45% remaining)
Processed datadata_2016MagD

Processing files: 100%|██████████| 1625/1625 [12:02<00:00,  2.25it/s]


Processed datadata_2016MagDown_0.root: 97 events before, 76 after (78.35% remaining)
Processed datadata_2016MagDown_1.root: 147 events before, 102 after (69.39% remaining)
Processed datadata_2016MagDown_10.root: 139 events before, 90 after (64.75% remaining)
Processed datadata_2016MagDown_100.root: 136 events before, 90 after (66.18% remaining)
Processed datadata_2016MagDown_1000.root: 101 events before, 65 after (64.36% remaining)
Processed datadata_2016MagDown_1001.root: 111 events before, 77 after (69.37% remaining)
Processed datadata_2016MagDown_1002.root: 138 events before, 99 after (71.74% remaining)
Processed datadata_2016MagDown_1003.root: 129 events before, 93 after (72.09% remaining)
Processed datadata_2016MagDown_1004.root: 70 events before, 48 after (68.57% remaining)
Processed datadata_2016MagDown_1005.root: 96 events before, 67 after (69.79% remaining)
Processed datadata_2016MagDown_1006.root: 92 events before, 56 after (60.87% remaining)
Processed datadata_2016MagDown_10

...

Processed datadata_2017MagDown_0.root: 96 events before, 65 after (67.71% remaining)
Processed datadata_2017MagDown_1.root: 150 events before, 89 after (59.33% remaining)
Processed datadata_2017MagDown_10.root: 144 events before, 99 after (68.75% remaining)
Processed datadata_2017MagDown_100.root: 144 events before, 101 after (70.14% remaining)
Processed datadata_2017MagDown_1000.root: 152 events before, 108 after (71.05% remaining)
Processed datadata_2017MagDown_1001.root: 139 events before, 99 after (71.22% remaining)
Processed datadata_2017MagDown_1002.root: 169 events before, 110 after (65.09% remaining)
Processed datadata_2017MagDown_1003.root: 156 events before, 112 after (71.79% remaining)
Processed datadata_2017MagDown_1004.root: 114 events before, 80 after (70.18% remaining)
Processed datadata_2017MagDown_1005.root: 146 events before, 96 after (65.75% remaining)
Processed datadata_2017MagDown_1006.root: 144 events before, 96 after (66.67% remaining)
Processed datadata_2017MagD

Processing files: 100%|██████████| 1364/1364 [10:15<00:00,  2.22it/s]


Processed datadata_2017MagUp_0.root: 105 events before, 65 after (61.90% remaining)
Processed datadata_2017MagUp_1.root: 168 events before, 106 after (63.10% remaining)
Processed datadata_2017MagUp_10.root: 141 events before, 76 after (53.90% remaining)
Processed datadata_2017MagUp_100.root: 121 events before, 82 after (67.77% remaining)
Processed datadata_2017MagUp_1000.root: 149 events before, 94 after (63.09% remaining)
Processed datadata_2017MagUp_1001.root: 146 events before, 83 after (56.85% remaining)
Processed datadata_2017MagUp_1002.root: 173 events before, 106 after (61.27% remaining)
Processed datadata_2017MagUp_1003.root: 99 events before, 55 after (55.56% remaining)
Processed datadata_2017MagUp_1004.root: 163 events before, 104 after (63.80% remaining)
Processed datadata_2017MagUp_1005.root: 143 events before, 92 after (64.34% remaining)
Processed datadata_2017MagUp_1006.root: 136 events before, 75 after (55.15% remaining)
Processed datadata_2017MagUp_1007.root: 137 events

Processing files: 100%|██████████| 1364/1364 [10:10<00:00,  2.23it/s]

Processed datadata_2017MagUp_0.root: 89 events before, 64 after (71.91% remaining)
Processed datadata_2017MagUp_1.root: 126 events before, 87 after (69.05% remaining)
Processed datadata_2017MagUp_10.root: 113 events before, 68 after (60.18% remaining)
Processed datadata_2017MagUp_100.root: 125 events before, 83 after (66.40% remaining)
Processed datadata_2017MagUp_1000.root: 135 events before, 98 after (72.59% remaining)
Processed datadata_2017MagUp_1001.root: 115 events before, 77 after (66.96% remaining)
Processed datadata_2017MagUp_1002.root: 163 events before, 107 after (65.64% remaining)
Processed datadata_2017MagUp_1003.root: 78 events before, 56 after (71.79% remaining)
Processed datadata_2017MagUp_1004.root: 126 events before, 84 after (66.67% remaining)
Processed datadata_2017MagUp_1005.root: 109 events before, 67 after (61.47% remaining)
Processed datadata_2017MagUp_1006.root: 122 events before, 91 after (74.59% remaining)
Processed datadata_2017MagUp_1007.root: 148 events be